# Guide: **Understanding search methods to Isolate Path Effects**

## Isolating Path Effect for Latent Circuit Identidentification


**IPE (Isolating Path Effects)** is a novel method for **mechanistic interpretability** that identifies entire computational paths in large language models. 
Unlike common approaches that focus on individual edges, IPE uses an efficient top-down search to find paths from input embeddings to output logits that significantly affect model predictions. The method precisely measures a path's contribution by selectively modifying messages along the residual stream, allowing for both **path ablation** and **counterfactual replacement** while avoiding interference from other paths. 

<p align="center">
    <img src="./images/path.png" alt="Isolating Path Effects" width="250vh"><br>
    <text>Figure 1: An example of a path leading from the first token embedding to the residual associated with the final prediction </text>
</p>


At the core of the IPE code there are multiple distinct search strategies to efficiently find relevant computational paths. All strategies operate as a backward search, starting from the final model output and moving layer by layer back towards the input.

## Shared Logic: 

Each strategy shares a common framework, which is the reason we collect all these search methods under the umbrella of IPE:

 - Path Representation: Paths are represented as a list of Node objects, with each Node representing a component (e.g., attention heads, MLPs, Embedding) at a particular layer and position. Attention nodes may correspond to single heads and gather information from a residual in a previous position (key-value position)

 - Path Evaluation: The evaluation function is the most critical component. It takes a complete or partial path and calculates its contribution to the model's output using a user-defined metric. This is where the core "path isolation" logic happens, modifying the residual stream messages.

 - Backward Search: All metrics evaluates how the removal of the contribution of a path from the final residual would affect the model prediction. The subsequent need to evaluate up to the final node makes approaching the search starting from the final residual down to the input natural.

 - Pruning: The search space is pruned based on the calculated contribution. Paths that don't meet a certain criterion (threshold, top-n candidates, BFS) are discarded to maintain computational efficiency. Note that the number of space is exponential in the number of layer, with base proportional to the number of heads times the number of position, therefore it is unfeasible to perform an exhaustive search.

## Differenes:

We present two different types of algorithm differentiated by the path evaluation modality, where the second method uses a linear approximation of the first:
 -Path Message Patching
 - Path Attribution Patching

Furthermore, as previously highlighted, different pruning methodologies yields slightly different search approaches (threshold, top-n candidates, BFS).
In conclusion 6 different search function to Isolate Path Effect are present in the IPE library.


## **Path Message Patching**

Path Message Patching is the core method for isolating and quantifying the effect of a specific path. It works by performing a modified forward pass along the selected path, note that this forward pass involves only the nodes in the path, rather than the entire model. Instead of simply zeroing out an activation, which would affect all downstream paths, this method precisely isolates the contribution of the targeted path.

Before discussing path message patching, it is necessary to clarify some concepts: path, message, and node input.

<p align="center">
    <div style="display:flex; gap:1.5rem; justify-content:center; align-items:flex-start; flex-wrap:wrap;">
        <figure style="text-align:center; margin:0;">
            <img src="./images/path.png" alt="Isolating Path Effects" style="width:12rem; height:auto; max-width:40vw;">
            <figcaption style="font-size:0.9rem; margin-top:0.25rem;">(a)</figcaption>
        </figure>
        <figure style="text-align:center; margin:0;">
            <img src="./images/inputs.png" alt="Inputs" style="width:12rem; height:auto; max-width:40vw;">
            <figcaption style="font-size:0.9rem; margin-top:0.25rem;">(b)</figcaption>
        </figure>
        <figure style="text-align:center; margin:0;">
            <img src="./images/path_message.png" alt="Inputs" style="width:12rem; height:auto; max-width:40vw;">
            <figcaption style="font-size:0.9rem; margin-top:0.25rem;">(c)</figcaption>
        </figure>
    </div>
    <text>Figure 2: (a) An example of a path leading from the first token embedding to the residual associated with the final prediction. (b) Visualization of a node input as the sum of the contributions from all its predecessors. (c) Visualization of the path message calculation as the effect of propagating an ablation through the path (note that X can be either zeros for ablation or a value for the counterfactual).</text>
</p>

### Key Definitions

Some important definitions for understanding the method:

 - **Node:** A node is a model component. Node types include embeddings, attention heads, MLPs, and the final residual. Each node is characterized by the position where its output is written and the layer it belongs to. For attention, nodes can be further specified by the key-value position and the specific head.

 - **Node Input:** The input to a node is the residual stream value at that point in the forward pass.

 - **Message:** The message from node A to node B is the portion of B's input attributable to the output of node A. A node's input is the sum of all its predecessors' outputs.

 - **Path:** A path is a sequence of consecutive nodes in the computational tree representing the model.

### Path Message

Message calculation is performed through an iterative, bottom-up procedure (from the first node in the path to the last):

 1. **Initial Message Removal:** Starting with the first node in the path, compute the change (∆$m_i$) as the difference between the output from the first node under the counterfactual input (or zero, for ablation) and the output under the original input.

 2. **Iterative Propagation:** Next, compute the variation in the message of the subsequent component caused by patching the previous node. The next variation (∆$m_{i+1}$) is calculated as the difference in the output of $node_{n+1}$ when its input is varied by adding ∆$m_i$, compared to the output under the original input.

 3. **Path Message:** The variation in the model's final residual attributable to the path is the message variation obtained for the last path node.

This approach enables both path ablation (setting the path's contribution to zero) and path counterfactual replacement (replacing the path's effect with what would have been produced by a different input), simply by changing the initial message removal method.

### Evaluation

Once the path message is computed, various metrics can be used to evaluate the effect of removing it. For example, the `target_logit_percentage` metric scores a path using the formula $Path\_Score = (logit(unembed(resid)) - logit(unembed(resid - ∆m_{path})))/logit(resid)$, where logit is the logit score of the target token and unembed performs the final layernorm and unembedding.

It is important to note that all metrics are defined by removing a message from the final residual stream. Consequently, a path must terminate at the final residual node to produce a contribution score. The path's start, however, can be anywhere (it does not have to be an embedding), so partial paths beginning mid-model can be evaluated. Paths that stop before the final residual node cannot be fully evaluated. This is why a top-down search approach is used.

### **Path Message Patching with Threshold**

The baseline algorithm for finding the most relevant paths is as follows:

1. Start with a frontier containing a path of length zero, composed only of the `FINAL_Node`, a dummy node representing the final residual stream.
2. For each path in the frontier, evaluate all possible continuations by extending the path with a new node.
3. For each expansion whose contribution exceeds a specified threshold (optionally using the absolute value), either add it to the frontier if it is incomplete, or add it to the list of relevant complete paths.
4. Repeat step 2 until the frontier is empty.

The actual implementation includes two further approximations for evaluating expansions involving attention:

1. **batch_positions:** Evaluate the relevance of individual key-value positions only after confirming that altering all positions simultaneously is relevant.
2. **batch_heads:** Evaluate the relevance of individual heads only after confirming that the contribution of the entire attention module is relevant.

Below is an example of how to invoke the path message patching with threshold function.


In [ ]:
from transformer_lens import HookedTransformer
from ipe.metrics import target_logit_percentage
from ipe.graph_search import  PathMessagePatching
from ipe.nodes import FINAL_Node
from functools import partial

model = HookedTransformer.from_pretrained("gpt2-small")
_, cache = model.run_with_cache("When John and Mary went to the shop, John gave the bags to")
target = " Mary"
metric = partial(target_logit_percentage, model=model, clean_final_resid=cache['blocks.11.hook_resid_post'], target_tokens=[model.to_single_token(target)])

paths =  PathMessagePatching(
    model=model,
    metric=metric,
    root=FINAL_Node(
        model=model,
        metric=metric,
        layer=11,
        position=cache['blocks.11.hook_resid_post'].shape[1] - 1,
        msg_cache=dict(cache), # Message cache, it must be a dictionary
        cf_cache={}, # Counterfactual cache, if you want to use a different cf input than the clean one
        patch_type='zero' # Patch type, can be 'zero' or 'counterfactual', if 'counterfactual' cf_cache must be provided
    ),
    min_contribution = 0.5, # Minimum contribution for a path to be considered
    include_negative = True, # Wether to include paths with negative contribution
    return_all = False, # Wether to return all paths found or only the highly contributing ones
    batch_positions = True, # Wether to first evaluate the effect of patching all positions at one, then one by one
    batch_heads = True # Wether to first evaluate the effect of patching all heads at one, then one by one
)

Loaded pretrained model gpt2-small into HookedTransformer
(total 1)    Frontier: [(tensor(91.0508, grad_fn=<MeanBackward0>), [FINAL_Node(layer=11, position=14)])]


100%|██████████| 1/1 [00:01<00:00,  1.88s/it]


(total 79)    Frontier: [(tensor(14.3535, grad_fn=<MeanBackward0>), [ATTN_Node(layer=9, head=9, position=14, keyvalue_position=None, patch_query=True, patch_key=False, patch_value=False), FINAL_Node(layer=11, position=14)]), (tensor(14.1150, grad_fn=<MeanBackward0>), [ATTN_Node(layer=9, head=9, position=14, keyvalue_position=4, patch_query=False, patch_key=True, patch_value=True), FINAL_Node(layer=11, position=14)])]... ]


100%|██████████| 79/79 [00:29<00:00,  2.71it/s]


(total 707)    Frontier: [(tensor(28.7930, grad_fn=<MeanBackward0>), [ATTN_Node(layer=11, head=0, position=14, keyvalue_position=None, patch_query=True, patch_key=False, patch_value=False), MLP_Node(layer=11, position=14), FINAL_Node(layer=11, position=14)]), (tensor(10.4667, grad_fn=<MeanBackward0>), [ATTN_Node(layer=11, head=8, position=14, keyvalue_position=None, patch_query=True, patch_key=False, patch_value=False), MLP_Node(layer=11, position=14), FINAL_Node(layer=11, position=14)])]... ]


...

100%|██████████| 16/16 [00:02<00:00,  6.69it/s]


(total 1)    Frontier: [(tensor(0.5339, grad_fn=<MeanBackward0>), [MLP_Node(layer=0, position=10), MLP_Node(layer=2, position=10), ATTN_Node(layer=3, head=0, position=10, keyvalue_position=None, patch_query=True, patch_key=False, patch_value=False), MLP_Node(layer=3, position=10), MLP_Node(layer=4, position=10), ATTN_Node(layer=6, head=9, position=10, keyvalue_position=None, patch_query=True, patch_key=False, patch_value=False), ATTN_Node(layer=8, head=6, position=14, keyvalue_position=10, patch_query=False, patch_key=True, patch_value=True), ATTN_Node(layer=9, head=9, position=14, keyvalue_position=None, patch_query=True, patch_key=False, patch_value=False), FINAL_Node(layer=11, position=14)])]


100%|██████████| 1/1 [00:00<00:00,  1.15it/s]


(total 2)    Frontier: [(tensor(1.1124, grad_fn=<MeanBackward0>), [ATTN_Node(layer=0, head=9, position=10, keyvalue_position=None, patch_query=True, patch_key=False, patch_value=False), MLP_Node(layer=0, position=10), MLP_Node(layer=2, position=10), ATTN_Node(layer=3, head=0, position=10, keyvalue_position=None, patch_query=True, patch_key=False, patch_value=False), MLP_Node(layer=3, position=10), MLP_Node(layer=4, position=10), ATTN_Node(layer=6, head=9, position=10, keyvalue_position=None, patch_query=True, patch_key=False, patch_value=False), ATTN_Node(layer=8, head=6, position=14, keyvalue_position=10, patch_query=False, patch_key=True, patch_value=True), ATTN_Node(layer=9, head=9, position=14, keyvalue_position=None, patch_query=True, patch_key=False, patch_value=False), FINAL_Node(layer=11, position=14)]), (tensor(0.6922, grad_fn=<MeanBackward0>), [ATTN_Node(layer=0, head=11, position=10, keyvalue_position=None, patch_query=True, patch_key=False, patch_value=False), MLP_Node(laye

100%|██████████| 2/2 [00:00<00:00, 35.46it/s]


In [7]:
print(f"Found {len(paths)} paths with contribution > 0.5")

Found 388 paths with contribution > 0.5


### **Path Message Patching with Best First Search**

The **Best First Search** (BFS) style of search for finding the most relevant paths operates with a slightly different prioritization than a threshold-based approach. Instead of exploring all paths that meet a minimum contribution, it focuses on the most promising paths at each step.

1.  Start with a **priority queue** containing a path of length zero, composed only of the `FINAL_Node`. The priority of this initial path is set to a high value (e.g., the full model's logit difference).
2.  While the priority queue is not empty, pop the path with the highest current contribution.
3.  Evaluate all possible continuations by extending the path with a new node.
4.  For each new expansion, calculate its contribution and push it onto the priority queue.
5.  If a path reaches the `EMBEDDING_Node`, it is considered a complete path and is added to the list of relevant paths.
6.  The search continues until the priority queue is empty or a predefined number of paths have been found.

Like the threshold-based approach, this method also incorporates approximations to handle the complexity of attention modules, specifically **batching positions** and **batching heads** to efficiently identify promising candidates before evaluating individual components. The key difference is that BFS guarantees that the paths with the highest scores are explored first, making it well-suited for finding the top-k most important paths without needing to set a specific threshold.

In [ ]:
from ipe.graph_search import PathMessagePatching_BestFirstSearch
paths =  PathMessagePatching_BestFirstSearch(
    model=model,
    metric=metric,
    root=FINAL_Node(
        model=model,
        metric=metric,
        layer=11,
        position=cache['blocks.11.hook_resid_post'].shape[1] - 1,
        msg_cache=dict(cache), # Message cache, it must be a dictionary
        cf_cache={}, # Counterfactual cache, if you want to use a different cf input than the clean one
        patch_type='zero' # Patch type, can be 'zero' or 'counterfactual', if 'counterfactual' cf_cache must be provided
    ),
    top_n=300, # Number of nodes to find
    max_time=600, # Maximum time in seconds to run the search
    include_negative = True, # Wether to include paths with negative contribution
    batch_positions = True, # Wether to first evaluate the effect of patching all positions at one, then one by one
    batch_heads = True # Wether to first evaluate the effect of patching all heads at one, then one by one
)


Completed paths:  60%|██████    | 181/300 [10:00<06:34,  3.32s/it]


In [10]:
print(f"Found {len(paths)} relevant paths")

Found 181 relevant paths


### **Path Message Patching with Limited Level Width**

The Limited Level Width approach is based on a Breadth First Search, similarly to the thresholding approach. However, instead of adding to the frontier only nodes achieving a minimum contribution, the top k candidate incomplete paths are added to the frontier.

1. Start with a frontier containing a path of length zero, composed only of the `FINAL_Node`, a dummy node representing the final residual stream.
2. For each path in the frontier, evaluate all possible continuations by extending the path with a new node.
3. Whenever a path is completed add it to the list of completed paths.
4. Once all path in the frontier has been expanded keep the k ones having the highest contribution that are not completed.
5. Repeat from step 2 until the frontier is empty.

The actual implementation includes two further approximations for evaluating expansions involving attention:

1. **batch_positions:** Evaluate the relevance of individual key-value positions only after confirming that altering all positions simultaneously is relevant.
2. **batch_heads:** Evaluate the relevance of individual heads only after confirming that the contribution of the entire attention module is relevant.

Below is an example of how to invoke the path message patching with threshold function.


In [12]:
from ipe.graph_search import PathMessagePatching_LimitedLevelWidth
paths =  PathMessagePatching_LimitedLevelWidth(
    model=model,
    metric=metric,
    root=FINAL_Node(
        model=model,
        metric=metric,
        layer=11,
        position=cache['blocks.11.hook_resid_post'].shape[1] - 1,
        msg_cache=dict(cache), # Message cache, it must be a dictionary
        cf_cache={}, # Counterfactual cache, if you want to use a different cf input than the clean one
        patch_type='zero' # Patch type, can be 'zero' or 'counterfactual', if 'counterfactual' cf_cache must be provided
    ),
    max_width=200, # Maximum number of nodes to expand at each level
    include_negative = True, # Wether to include paths with negative contribution
    batch_positions = True, # Wether to first evaluate the effect of patching all positions at one, then one by one
    batch_heads = True # Wether to first evaluate the effect of patching all heads at one, then one by one
)


Expanding level (size 200): 100%|██████████| 200/200 [00:04<00:00, 43.14it/s]


In [13]:
print(f"Found {len(paths)} paths")

Found 2201 paths


## **Path Attribution Patching**


Path Attribution Patching is a **linear approximation** of the more computationally intensive **Path Message Patching**. This method leverages the **chain rule**, a core principle of backpropagation, to efficiently estimate a path's contribution to the final output. The key insight is that the gradient of the loss with respect to a specific activation can be decomposed into the sum of contributions from all computational paths leading from that activation to the final loss.

Instead of performing a full modified forward pass, Path Attribution Patching approximates a path's contribution as the product of the change in an edge's activation and the gradient of the final path output with respect to that edge. 
It is important to underline how the gradient along the path output is different from the gradient of the output with respect to the activation. To briefly explain the difference we can look at Figure 3. 
In the image we can see the computational graph of a simple function *F* of *x*:
$$
    F(x) = f(g(x) + h(x) + x)
$$
The gradient of the output *F(x)* with respect to *x* is the following:

$$
    \frac{\partial F}{\partial x} = \frac{\partial F(x)}{\partial g(x)} \frac{\partial g(x)}{\partial x} + \frac{\partial F(x)}{\partial h(x)} \frac{\partial h(x)}{\partial x} + \frac{\partial F(x)}{\partial x}
$$

We want to focus our attention of how this gradient is divided into three different terms, each corresponding to a path leading the input *x* up to the final node output. We refer to these as **gradients along the path**
In general it holds that:
The gradient of the output with respect to an intermediate activation is the sum of all gradients along each different path leading the activation to the final output:
$$
\frac{\partial F}{\partial x} = \sum_{path \in \text{paths }} \left( \prod_{node \in path} \frac{\partial out_{node}}{\partial in_{node}} \right)
$$
*where each term in the sum corresponds to the gradient along a specific computational path $p$ from $x$ to the output.*

Note that the gradient along a single path can be computed by the product of the gradients $\frac{\partial out_i}{\partal in_i} of the output of a componet $out_i$ with respect to its input $in_i$ for each node in the path (due to the backpropagation rule).


<p align="center">
    <div style="display:flex; gap:1.5rem; justify-content:center; align-items:flex-start; flex-wrap:wrap;">
        <figure style="text-align:center; margin:0;">
            <img src="./images/path.png" alt="Isolating Path Effects" style="width:12rem; height:auto; max-width:40vw;">
            <figcaption style="font-size:0.9rem; margin-top:0.25rem;">(a)</figcaption>
        </figure>
        <figure style="text-align:center; margin:0;">
            <img src="./images/inputs.png" alt="Inputs" style="width:12rem; height:auto; max-width:40vw;">
            <figcaption style="font-size:0.9rem; margin-top:0.25rem;">(b)</figcaption>
        </figure>
        <figure style="text-align:center; margin:0;">
            <img src="./images/path_message.png" alt="Inputs" style="width:12rem; height:auto; max-width:40vw;">
            <figcaption style="font-size:0.9rem; margin-top:0.25rem;">(c)</figcaption>
        </figure>
    </div>
    <text>Figure 2: (a) An example of a path leading from the first token embedding to the residual associated with the final prediction. (b) Visualization of a node input as the sum of the contributions from all its predecessors. (c) Visualization of the path message calculation as the effect of propagating an ablation through the path (note that X can be either zeros for ablation or a value for the counterfactual).</text>
</p>


### Path Evaluation via Attribution

Given a path it's **Attribution Score** is computed as follows:

 1. **Initial Message Change **: Starting with the first node in the path, compute the change (∆$m_{initial}$) as the difference between the output from the first node under the counterfactual input (or zero, for ablation) and the output under the original input. Note that these values can be easily cached by saving the activation of the clean and corrupted forward passes.

    $$ \Delta m^{initial} = in^{initial}_{corr} - in^{initial}_{clean} $$
    where $in^{initial}$ is the input associated with the first node int the path

 2. **Gradient Along the Path**: Compute the gradient of the output with respect to the path initial message as:

    $$\nabla _{path} out = \frac{\partial out}{\partial path} = \frac{\partial out_{path}}{\partial path} = \prod_{node \in path} \frac{\partial out_{node}}{\partial in_{node}}$$

    Note that an equivalent formula applies if we are analyzing a metric $L$ which is a function in the model output:

    $$\nabla _{path} L(out) = \frac{\partial L(out)}{\partial path} = \frac{\partial L(out)}{\partial out} \cdot \prod_{node \in path} \frac{\partial out_{node}}{\partial in_{node}}$$

    Note that the computation of the gradient along a path obtained by adding a node ($n0$) to an incomplete path ($path$) is relatively fast if we already have computed $\nabla _p L(out)$, it just require a backward pass on the single component $n0$:

    $$\nabla _{(candidate \cup path)} L(out) =  \frac{\partial L(out)}{\partial path} \cdot  \frac{\partial out_{n0}}{\partial in_{n0}} $$

 3. **Path Attribution Score**: The final score for a path is approximated by the dot product of the cached initial message change and the path gradient. This can be expressed using a first order Taylor expansion as:

    $$L(out_{clean} | Do(path_{clean} = path_{corr})) \approx L(out_{clean}) + (\Delta m_{initial})^T \nabla _{path} L(out) $$


### Efficiency gain

At a first glance this method of evaluating a path is not faster than directly computing the value of $L(out_{clean} | Do(path_{clean} = path_{corr})) - L(out_{clean})$. This is true when we have to evaluate a single path, because the approximated method requires a custom backward pass along the path which is computantionally slightly more expensive than computing the forward pass along the path.

Nonetheless, during our backward search, when we want to expand an incomplete path, we need to calculate the effect of thousands of possible continuations of it. 
When using Path Message Passing this leads to a new forward pass along each candidate path.
On the other hand the evaluation of all possible expansions using Path Attribution Patching is much faster. 
Particularly the initial message is actually obtainable as the sum of all the activations on the incoming edges, therefore by isolating them we can compute an attribution score for each of the possible path extension.

$$ \Delta m^{initial} = in^{initial}_{corr} - in^{initial}_{clean} = \sum_{candidate \in predecessors} out^{candidate}_{corr} - \sum_{candidate \in predecessors} out^{candidate}_{clean} $$

So isolating the message due to a contribution of the edge coming from a single candidate child:

$$ \Delta m^{from candiate} = out^{candidate}_{corr} - out^{candidate}_{clean} $$

The product between this variation in the input message and the gradient along the path yields the score of the path attributable to the edge linking to the candidate.

$$L(out_{clean} | Do((candidate \cup path)_{clean} = (candidate \cup path)_{corr})) \approx L(out_{clean}) + (\Delta m_{from candidate})^T \nabla _{path} L(out) $$

After calculating the gradient along the path, the cost of calculating the attribution score of each new contribution is just the one of computing a dot product (note that the edge messages can be easily cached in just two initial forward passes).


### Implementation

As for Path Message Patching the mechanism for evaluating the path allows for the same backward search methodologies. However due to the speed of evaluation of candidate expansion there is no need for further heuristics such as first evaluating the contribution from all positions and/or all heads before addressing the contribution of a single component.

We want to highlight how once again the search starting from the final node, moving backward to the initial nodes is more computationally efficient because the output can gradient computation is a vector $\cdot$ matrix multiplication in this case, while when going forward it would require matrix $\cdot$ matrix multiplications. This is simply caused by the fact that the gradient $\frac{\partial L(out)}{\partial out}$ is a vector, while for a generic node $\frac{\partial node_{out}}{\partial node_{in}}$ is a Jacobian matrix (of size d_model x d_model, so unfeasible to be cached)



### **Path Attribution Patching with Threshold**

The baseline algorithm for finding the most relevant paths is equivalent to the corresponding Path Message Patching implementation:

1. Start with a frontier containing a path of length zero, composed only of the `FINAL_Node`, a dummy node representing the final residual stream.
2. For each path in the frontier, evaluate all possible continuations by extending the path with a new node.
3. For each expansion whose contribution exceeds a specified threshold (optionally using the absolute value), either add it to the frontier if it is incomplete, or add it to the list of relevant complete paths.
4. Repeat step 2 until the frontier is empty.

Below is an example of how to invoke the path message patching with threshold function.


In [ ]:
from transformer_lens import HookedTransformer
from ipe.metrics import target_logit_percentage
from ipe.graph_search import  PathAttributionPatching
from ipe.nodes import FINAL_Node
from functools import partial

model = HookedTransformer.from_pretrained("gpt2-small")
_, cache = model.run_with_cache("When John and Mary went to the shop, John gave the bags to")
target = " Mary"
metric = partial(target_logit_percentage, model=model, clean_final_resid=cache['blocks.11.hook_resid_post'], target_tokens=[model.to_single_token(target)])

minimum_contribution_threshold = 2

paths =  PathAttributionPatching(
    model=model,
    metric=metric,
    root=FINAL_Node(
        model=model,
        metric=metric,
        layer=11,
        position=cache['blocks.11.hook_resid_post'].shape[1] - 1,
        msg_cache=dict(cache), # Message cache, it must be a dictionary
        cf_cache={}, # Counterfactual cache, if you want to use a different cf input than the clean one
        patch_type='zero' # Patch type, can be 'zero' or 'counterfactual', if 'counterfactual' cf_cache must be provided
    ),
    min_contribution = 0.5, # Minimum contribution for a path to be considered
    include_negative = True, # Wether to include paths with negative contribution
    confirm_relevance = False, # If true this became an hybrid approach, attribution selects relevant candidate expansions, then message patching confirms them before adding them to the path
)

Loaded pretrained model gpt2-small into HookedTransformer


100%|██████████| 5/5 [00:00<00:00, 233.78it/s]


In [5]:
print(f"Found {len(paths)} paths")
actually_important_paths = [(c, p) for c, p in paths if abs(c) > 0.5]
print(f"Found {len(actually_important_paths)} paths with contribution > 0.5")

Found 990 paths
Found 537 paths with contribution > 0.5


### **Path Attribution Patching with Best First Search**

The **Best First Search** (BFS) style of search for finding the most relevant paths operates with a slightly different prioritization than a threshold-based approach. Instead of exploring all paths that meet a minimum contribution, it focuses on the most promising paths at each step.

1.  Start with a **priority queue** containing a path of length zero, composed only of the `FINAL_Node`. The priority of this initial path is set to a high value (e.g., the full model's logit difference).
2.  While the priority queue is not empty, pop the path with the highest current contribution.
3.  Evaluate all possible continuations by extending the path with a new node.
4.  For each new expansion, calculate its contribution and push it onto the priority queue.
5.  If a path reaches the `EMBEDDING_Node`, it is considered a complete path and is added to the list of relevant paths.
6.  The search continues until the priority queue is empty or a predefined number of paths have been found.


In [ ]:
from ipe.graph_search import  PathAttributionPatching_BestFirstSearch


paths =  PathAttributionPatching_BestFirstSearch(
    model=model,
    metric=metric,
    root=FINAL_Node(
        model=model,
        metric=metric,
        layer=11,
        position=cache['blocks.11.hook_resid_post'].shape[1] - 1,
        msg_cache=dict(cache), # Message cache, it must be a dictionary
        cf_cache={}, # Counterfactual cache, if you want to use a different cf input than the clean one
        patch_type='zero' # Patch type, can be 'zero' or 'counterfactual', if 'counterfactual' cf_cache must be provided
    ),
    top_n = 100, # Minimum contribution for a path to be considered
    max_time = 900, # Maximum time in seconds to run the search
    include_negative = True, # Wether to include paths with negative contribution
)

### **Path Attribution Patching with Limited Level Width**

The Limited Level Width approach is based on a Breadth First Search, similarly to the thresholding approach. However, instead of adding to the frontier only nodes achieving a minimum contribution, the top k candidate incomplete paths are added to the frontier.

1. Start with a frontier containing a path of length zero, composed only of the `FINAL_Node`, a dummy node representing the final residual stream.
2. For each path in the frontier, evaluate all possible continuations by extending the path with a new node.
3. Whenever a path is completed add it to the list of completed paths.
4. Once all path in the frontier has been expanded keep the k ones having the highest contribution that are not completed.
5. Repeat from step 2 until the frontier is empty.

Below is an example of how to invoke the path message patching with threshold function.


In [ ]:
from ipe.graph_search import  PathAttributionPatching_LimitedLevelWidth


paths =  PathAttributionPatching_LimitedLevelWidth(
    model=model,
    metric=metric,
    root=FINAL_Node(
        model=model,
        metric=metric,
        layer=11,
        position=cache['blocks.11.hook_resid_post'].shape[1] - 1,
        msg_cache=dict(cache), # Message cache, it must be a dictionary
        cf_cache={}, # Counterfactual cache, if you want to use a different cf input than the clean one
        patch_type='zero' # Patch type, can be 'zero' or 'counterfactual', if 'counterfactual' cf_cache must be provided
    ),
    max_width = 100, # Maximum width of the search
    include_negative = True, # Wether to include paths with negative contribution
)